Spam-Datensatz in CSV-Datei,die letzte Spalte gibt an, ob die entsprechende E-Mail als Spam angesehen wird. Wir teilen den Datensatz in 80% Trainingsdaten und 20% Testdaten ein.
Es wird einen binären Klassifikator mittels Fischer-Diskriminante implementiert, und die Klassifikationsgenauigkeit als Ergebnis (output) gegeben.

In [20]:
import numpy as np

def file_get_contents(filename):
    return np.genfromtxt(filename, delimiter=',')

data = file_get_contents('resources/spambase.data')

Just take a look of the data.

In [21]:
print(data[:100])

[[0.000e+00 6.400e-01 6.400e-01 ... 6.100e+01 2.780e+02 1.000e+00]
 [2.100e-01 2.800e-01 5.000e-01 ... 1.010e+02 1.028e+03 1.000e+00]
 [6.000e-02 0.000e+00 7.100e-01 ... 4.850e+02 2.259e+03 1.000e+00]
 ...
 [0.000e+00 4.300e-01 4.300e-01 ... 6.100e+01 2.220e+02 1.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.700e+01 1.910e+02 1.000e+00]
 [1.240e+00 4.100e-01 1.240e+00 ... 1.900e+01 1.140e+02 1.000e+00]]


Wir teilen die Daten in 80% Trainingdaten und 20% Testdaten ein.

In [22]:
def extract_training(data):
    length = int(0.8 * len(data))
    return data[:length][:1813], data[:length][1813:]  

def extract_test(data):
    test = np.copy(data)
    np.random.shuffle(test)  
    length = int(0.2 * len(test)) 
    return test[:length]

print(extract_training(data)[:100])

(array([[0.000e+00, 6.400e-01, 6.400e-01, ..., 6.100e+01, 2.780e+02,
        1.000e+00],
       [2.100e-01, 2.800e-01, 5.000e-01, ..., 1.010e+02, 1.028e+03,
        1.000e+00],
       [6.000e-02, 0.000e+00, 7.100e-01, ..., 4.850e+02, 2.259e+03,
        1.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 7.700e-01, ..., 6.200e+01, 2.580e+02,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 6.600e+01, 1.010e+02,
        1.000e+00],
       [0.000e+00, 3.100e-01, 4.200e-01, ..., 3.180e+02, 1.003e+03,
        1.000e+00]]), array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 5.000e+00, 5.902e+03,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.000e+00, 2.600e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 3.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 3.300e+01, 3.980e+02,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 3.000e+00,
        0.000e+00],
     

Dann bekommen wir unsere Traingdaten und Testdaten. Teilen wir noch die Traingdaten in zwei Klassen: spam und non-spam, um zu benuzten fürs Lernen.

In [23]:
training = extract_training(data)  # ist ein Tupel
test = extract_test(data)

def spam_class(data):
    return extract_training(data)[0]

def non_spam_class(data):
    return extract_training(data)[1]

Ein Paar Schwerpunkte sind:
1, Matrix invertierbar zu umwandeln, um Inverse Matrix zu bekommen.
2, Kovarianz Matrix rechnen.
3, Mittelpunkte von jeder Klasse rechnen.

In [24]:
def get_it_invertible(matrix): # erzeuge einen invertierbaren Matrix
    a = 0.1
    id = np.identity(len(matrix))
    deter = np.linalg.det(matrix)

    while (deter == 0): # wenn die Determinante 0 ist, gibt's keinen Inverse Matrix
        matrix = np.add(matrix, np.dot(a, id))
        deter = np.linalg.det(matrix)
    return matrix


def mycovariance(matrix): # berechne die Kovarianzmatrix von unserer Trainingklasse
    middlevec = diameter(matrix)
    length = len(matrix)
    matrix1 = np.transpose(np.subtract(matrix, middlevec))
    matrix2 = np.transpose(matrix1)
    matrix = np.dot(matrix1, matrix2)
    return np.divide(matrix, length-1)


def diameter(images): # hier wird die Mittelvektor vom Matrix berechnet
    result = np.zeros((len(images[0])), dtype=np.float)

    for pixel in range(0, len(images[0])): # kriegen an jedem Index von allen Sample die durchschnittliche Werte.
        sum = 0

        for image in range(0, len(images)):
            sum += images[image][pixel]

        result[pixel] = (sum / len(images))

    return result

Das Hauptsache bei Fischer-Diskreminante ist die geeignete Projektionsrichtung zu finden. 
Mit "get_projection" Funktion können wir die projektion Funktion "f(x) = Wt*x + W0" rechnen, 
Wt ist die inverse Matrix von Projektion Koeffiziente, W0 ist eine Konstante, hier suchen wir so eine, dass mit der die Schätzung Richtigkeit vergrößert.

In [25]:
def projection_linear(data):    # hier ist unseres Linear Koeffiziente W
    covarianceSpam = mycovariance(spam_class(data))
    covarianceNonSpam = mycovariance(non_spam_class(data))
    covarianceSum = get_it_invertible(np.add(covarianceSpam, covarianceNonSpam))
    inverseSum = np.linalg.inv(covarianceSum)

    middleSpam = diameter(spam_class(data))
    middleNonSpam = diameter(non_spam_class(data))
    middleDiffer = np.subtract(middleSpam, middleNonSpam)

    return np.dot(inverseSum, middleDiffer)

def get_projection(data, testsample):
    spam = spam_class(data)
    middleSpam = diameter(spam)
    nonSpam = non_spam_class(data)
    middleNonspam = diameter(nonSpam)

    w1 = np.matrix(projection_linear(data))
    w2 = np.transpose(w1)

    a = np.dot(middleSpam, w2)
    b = np.dot(middleNonspam, w2)
    w0 = -0.95*(a + b)         # die Konstante W0, laut Erfahrung irgendwelche eingesetzt :(

    return np.dot(testsample, w2) + w0  # entspricht zu y = Wt*x+W0, y stellt jede Projektion dar

Mit "check_spam" Funktion wird geprüft, ob ein Testdatei laut gerechnete Projektion zur Spamklasse gehört.

In [26]:
def check_spam(testsample):
    data = file_get_contents('resources/spambase.data')
    result = get_projection(data, testsample)

    if (result > 0):  # Projektion positiv entspricht mit Spamklasse
        return 1
    else:
        return 0
    
print(check_spam(test[9]))
print(check_spam(test[37]))
print(check_spam(test[222]))
print(check_spam(test[-1]))

0
1
0
0


Jetzt schauen wir mal unseres Lernergebnis an.

In [32]:
def get_error_rate(data):
    testData = extract_test(data)
    error = 0

    for i in range (0, len(testData)):
        if (check_spam(testData[i]) != int(testData[i][57])):
            error += 1
    return (error / len(testData)) * 100

print('error rate is', get_error_rate(data), '%')

error rate is 7.282608695652174 %
